In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
pd.set_option('display.max_row', 111)
pd.set_option('display.max_column', 111)
data = pd.read_excel('dataset.xlsx')

In [3]:
df = data.copy()

In [4]:
missing_rate = df.isna().sum()/df.shape[0]

In [5]:
blood_columns = list(df.columns[(missing_rate<0.9)&(missing_rate>0.88)])
viral_columns = list(df.columns[(missing_rate < 0.80)&(missing_rate > 0.75)])

In [6]:
key_columns = ['Patient age quantile','SARS-Cov-2 exam result']

In [7]:
df = df[key_columns + blood_columns + viral_columns]#

In [8]:
df.shape

(5644, 33)

# Train_Test & Nettoyage & Encodage


In [9]:
from  sklearn.model_selection import train_test_split

In [10]:
trainset , testset = train_test_split(df, test_size=0.2, random_state=0)

In [11]:
trainset.shape

(4515, 33)

In [12]:
def encodage(df):
    code = {'negative':0,
            'positive':1,
            'not_detected':0,
            'detected':1}
    
    for col in df.select_dtypes('object').columns:
        df.loc[:,col] = df[col].map(code)
        
    return df

In [13]:
def feature_engineering(df):
    df['est malade'] = df[viral_columns].sum(axis=1) >= 1
    df = df.drop(viral_columns, axis=1)
    return df

In [14]:
def imputation(df):
    #df['is na'] = (df['Parainfluenza 3'].isna()) | (df['Leukocytes'].isna())
    #df = df.fillna(-999)
    df = df.dropna(axis=0)
    return  df

In [15]:
def preprocessing(df):
    
    df = encodage(df)
    df = feature_engineering(df)
    df = imputation(df)
    
    X = df.drop('SARS-Cov-2 exam result', axis=1)
    y = df['SARS-Cov-2 exam result']
    
    print(y.value_counts())
    
    return X, y

In [16]:
X_train, y_train= preprocessing(trainset) 
X_test, y_test = preprocessing(testset) 

0    422
1     65
Name: SARS-Cov-2 exam result, dtype: int64
0    95
1    16
Name: SARS-Cov-2 exam result, dtype: int64


In [21]:
y_train

543     0
203     0
1624    1
500     0
34      0
       ..
3299    0
5251    0
1071    0
4484    0
2008    0
Name: SARS-Cov-2 exam result, Length: 487, dtype: int64

In [18]:
DatasetTrain = pd.concat([X_train, y_train], axis=1)
DatasetTest = pd.concat([X_test, y_test], axis=1)

In [19]:
# Écrivez les données dans un fichier Excel
DatasetTrain.to_excel('DatasetTrain.xlsx', index=False)

In [20]:
# Écrivez les données dans un fichier Excel
DatasetTest.to_excel('DatasetTest.xlsx', index=False)